## Map Reduce Operations

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1569672546214_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

In [5]:
#Download "A Tale of Two Cities" as a string

import urllib.request
url = 'https://raw.githubusercontent.com/maigfrga/spark-streaming-book/master/data/books/tale2cities.txt'
response = urllib.request.urlopen(url)
data = response.read().decode('utf-8')
len(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

792969

In [6]:
lines = data.split('\n')
len(lines)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16273

In [7]:
book = sc.parallelize(lines)
book.first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\ufeffThe Project Gutenberg EBook of A Tale of Two Cities, by Charles Dickens\r'

In [14]:
#Clean the book of those grotesque special characters

def clean_line(line):
    return line.replace('\ufeff\r', '').\
        replace('\t', ' ').replace('\n', '').replace('\r', '').\
        replace('(', '').replace(')', '').replace("'", '').\
        replace('"', '').replace(',', ''). replace('.', '').\
        replace('*', '').replace('\ufeff', '')

clean_book = book.map(clean_line).filter(lambda x : x != '')
clean_book.first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'The Project Gutenberg EBook of A Tale of Two Cities by Charles Dickens'

In [17]:
#Split the book into a sequence of words

import re

def normalize_tokenize(line):
    return re.sub('\s+', ' ', line).strip().lower().split(' ')

tokens = clean_book.flatMap(normalize_tokenize)
tokens.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

138863

In [37]:
# Word count analysis

counts = tokens.filter(lambda x : len(x) > 3).map(lambda x : (x,1))
red_counts = counts.reduceByKey(lambda a, b : a + b)
tale_counts = red_counts.filter(lambda x : x[1] < 500)
tale_counts.takeOrdered(10, key=lambda x: -x[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('there', 492), ('been', 431), ('when', 425), ('which', 405), ('them', 360), ('would', 334), ('your', 328), ('into', 317), ('their', 317), ('lorry', 314)]

In [39]:
# Hamlet Word Count Analysis

url = 'https://raw.githubusercontent.com/maigfrga/spark-streaming-book/master/data/books/hamlet.txt'
lines = urllib.request.urlopen(url).read().decode('utf-8').split('\n')
book = sc.parallelize(lines)
tokens = book.map(clean_line).filter(lambda x : x != '').flatMap(normalize_tokenize)
ham_counts = tokens.filter(lambda x : len(x) > 3).map(lambda x : (x,1)).reduceByKey(lambda a, b : a + b).filter(lambda x : x[1] < 500)
ham_counts.takeOrdered(10, key=lambda x: -x[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('that', 368), ('this', 307), ('your', 265), ('with', 263), ('what', 202), ('haue', 173), ('lord', 170), ('king', 165), ('will', 154), ('then', 106)]

In [41]:
# Find shared words

common = ham_counts.join(tale_counts)
common.takeOrdered(10, key = lambda x : -1 * (x[1][0] + x[1][1]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('your', (265, 328)), ('there', (69, 492)), ('what', (202, 293)), ('when', (55, 425)), ('which', (62, 405)), ('been', (6, 431)), ('will', (154, 276)), ('them', (60, 360)), ('would', (69, 334)), ('their', (48, 317))]

In [42]:
# Words unique to Hamlet

ham_counts.leftOuterJoin(tale_counts).filter(lambda x : x[1][1] is None).take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('this', (307, None)), ('plays', (2, None)), ('readers', (3, None)), ('texts', (1, None)), ('shakespeare', (5, None))]

In [43]:
# Words unique to Tale

tale_counts.leftOuterJoin(ham_counts).filter(lambda x : x[1][1] is None).take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('ebook', (10, None)), ('dickens', (4, None)), ('anyone', (5, None)), ('anywhere', (5, None)), ('restrictions', (2, None))]